In [ ]:
from toydaq import Motor, Diode, scan_iter
import time
import tkinter as tk
from tkinter import ttk
from tkinter.messagebox import *

from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib import cm

from matplotlib import pyplot as plt

#----------------------

motors={"mot_m": Motor("mot_m", units='m'), "mot_nm": Motor("mot_nm", units='nm')}
diodes={"SIGNAL": Diode("SIGNAL"), "COUNTER": Diode("COUNTER") }

class Model():
   def __init__(self, motors, diodes):
        """
        Make motors, diodes, cameras available in the model with their methods and attributes.
        better in the future: load from a config file
        """
        self.motors = motors
        self.diodes = diodes
        self.cameras = {}

   def motor_names(self):
        """ Return names of all available motors """
        return [self.motors[label].name for label in self.motors]


class GUI():
   def __init__(self, model):
      """ Creates the main frame and links other frames to it. Only this layer should be visible to the model and controller. """

      self.model = model
      self._master = tk.Tk()

      self._frame = tk.Frame(self._master)
      self._frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)
      self.fig = Figure(figsize=(7.5, 4), dpi=80)
      self.ax0 = self.fig.add_subplot()

      self.canvas = FigureCanvasTkAgg(self.fig, master=self._frame)
      self.canvas.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=1)
      # Navigation toolbar for the canvas
      toolbar = NavigationToolbar2Tk(self.canvas, self._frame)
      toolbar.update()
      
      #plot button
      self.plotBut = tk.Button(self._frame, text="Plot 1D Scan")
      self.plotBut.pack(side="top", fill=tk.BOTH)
      self.plotBut.bind("<Button-1>", self.create_scan_plot)

      # Create a Comboox
      self.motor_sel_title=ttk.Label(self._frame, text='Motor selection').pack()
      self.motor_name=tk.StringVar() #Motornames are strings
      self.motor_selCombo=ttk.Combobox(self._frame, textvariable=self.motor_name)
      self.motor_selCombo['values']=self.model.motor_names()
      self.motor_selCombo['state'] = 'readonly'
      self.motor_selCombo.pack(side="top", fill=tk.BOTH)
      self.motor_selCombo.bind("<<ComboboxSelected>>", self.motor_selection)  

      #Create display for current motor values
      self.curr_motor_val=tk.StringVar()
      self.curr_motor_valLabel=ttk.Label(self._frame, textvariable=self.curr_motor_val)
      self.curr_motor_valLabel.pack(side="top", fill=tk.BOTH)



   def create_scan_plot(self, event):
      print(f"Hallo Gudrun, Hallo Welt")
      
      mot_selected=self.get_current_motor_selection()

      time.sleep(5)
 
      #now with the scan:
      for i, pos in enumerate(scan_iter(self.model.motors[mot_selected], 0, 10, 1, cb=None, show_progress=True)):
         print(pos)
         print("Motor is running")
        
         #now we try to send each motor value to the motor output label
         self.curr_motor_val.set(f"during scan {i} {self.model.motors[mot_selected]}")
         self._master.update()
         time.sleep(0.5)

      

   def get_current_motor_selection(self):
      #get_curent_motor
      print(self.motor_name.get(), type(self.motor_name.get()))   #string
      print(self.motor_selCombo.get(), type(self.motor_selCombo.get()))  #string

      mot_selected=self.motor_selCombo.get()

      #update value to the curr_motor_val label
      print('Update to ',self.model.motors[mot_selected])
      self.curr_motor_val.set(f"hallo update , {self.model.motors[mot_selected]}")
      self._master.update()
      return mot_selected

     

   def on_motor_selection(self, mot_selected):
      """ Sends information about current motor to the label """
      # shows the selected motor and its position in the label
      #self.sidepanel.curr_motor_valLabel.configure(text=self._model.motors[mot_selected])

      print('Here now in on_motor_selection')
      self.curr_motor_val.set(f"from on_motor_selection {self.model.motors[mot_selected]}")

      print(f"{mot_selected} was selected.")
      print('Returns', type(self.model.motors[mot_selected]), self.model.motors[mot_selected] )
         
      # return motor as object 
      return self.model.motors[mot_selected]

   def motor_selection(self, event):
      """ Selects motors on event action """
      # this method is directly bound to the function, so we can make use of this option
      mot_selected = event.widget.get()
      print('This is motor_selection ', type(mot_selected), mot_selected )
      
      #sends value to on_motor_selection
      return self.on_motor_selection(mot_selected) #returns string


   def motor_names(self):
      """ Return names of all available motors """
      return [self.model.motors[label].name for label in self.model.motors]

if __name__ == '__main__':

   model = Model(motors, diodes)
   
   # inject into Controller
   c = GUI(model)
   
    # start the GUI
   #c._master.mainloop()
   c._master.mainloop()